# Assignment Day 8

## Team members:
- Samuel Nebgen s6sanebg@uni-bonn.de
- Muhammad Humza Arain s27marai@uni-bonn.de
- Julian Meigen s82jmeig@uni-bonn.de

## 17.09.2025

Contributions were made by all team members in around the same amount, either based on discussions or coding.

# Task 1: Understand the dataset

## a) Load the dataset Hungarian Chickenpox

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [2]:
path_time_series = os.path.join("..", "data", "raw", "hungary_chickenpox.csv")
path_graph = os.path.join("..", "data", "raw", "hungary_county_edges.csv")

df_time = pd.read_csv(path_time_series, index_col=0, )
df_time.index = pd.to_datetime(df_time.index, format="%d/%m/%Y")

In [3]:
df_time.head()

,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,HEVES,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
Date,,,,,,,,,,,,,,,,,,,,
2005-01-03,168,79,30,173,169,42,136,120,162,36,130,57,2,178,66,64,11,29,87,68
2005-01-10,157,60,30,92,200,53,51,70,84,28,80,50,29,141,48,29,58,53,68,26
2005-01-17,96,44,31,86,93,30,93,84,191,51,64,46,4,157,33,33,24,18,62,44
2005-01-24,163,49,43,126,46,39,52,114,107,42,63,54,14,107,66,50,25,21,43,31
2005-01-31,122,78,53,87,103,34,95,131,172,40,61,49,11,124,63,56,7,47,85,60


In [4]:
from torch_geometric.data import Data
import torch

In [5]:
edge_list = torch.tensor(pd.read_csv(path_graph)[["id_1", "id_2"]].to_numpy(), dtype=torch.long)
edge_list[:3]

tensor([[ 0, 10],
        [ 0,  0],
        [ 0,  1]])

In [6]:
data = Data(x=df_time.values, edge_index=edge_list.t().contiguous())

## b) Identify the number of edges, nodes, and temporal features.

In [7]:
# Number of nodes:
data.num_nodes = data.x.shape[1]
print("Number of nodes: ", data.num_nodes)

# Number of edges:
print("Number of edges: ", data.num_edges)

# Number of temporal features per node:
data.num_temp_features = data.x.shape[0]
print("Number of temporal features per node: ", data.num_temp_features)

Number of nodes:  20
Number of edges:  102
Number of temporal features per node:  522


In [8]:
# Add additional information to the data object
data.dates = df_time.index.tolist() # Need to convert it back to a list because PyG does not support pd.DatetimeIndex
data.county_names = df_time.columns.tolist()

In [9]:
print(data)

Data(x=[522, 20], edge_index=[2, 102], num_nodes=20, num_temp_features=522, dates=[522], county_names=[20])


## c) Plot the adjacency matrix of the counties.

In [10]:
from torch_geometric.utils import to_dense_adj


In [11]:
adj_mat = to_dense_adj(data.edge_index, max_num_nodes=data.num_nodes).squeeze().numpy()

In [12]:
print(adj_mat)

[[1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.

# Task 2: Load the dataset into pytorch-geometric-temporal.

## a) Load the dataset with the class: ChickenpoxDatasetLoader

In [13]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader

/Users/julianmeigen/Uni/2_Semester/ML_BlockCourse/ML-handson/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
loader = ChickenpoxDatasetLoader(lags=4)
dataset = loader.get_dataset()

In [33]:
snapshot = dataset[0]
print(snapshot)

Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])


## b) Identify the differences between loading the raw dataset from Task 1 and the processed dataset from this task.

By loading the temporal dataset with ChickenpoxDatasetLoader(lags=4), the vertices are the lagged weekly count of chickenpox cases. That means that each node has 4 features which shows the Chickenpox cases at t, t-1, t-2 and t-4. 

The target y shows the Chickenpox case for each node (county) in the upcomming week t+1. 

The dataset has 517 different time-snapshots 

In [47]:
print(f"The temporal dataset has {dataset.snapshot_count} snapshots.")
print(f"Each snapshot has {snapshot.num_features} features per node. Which shows the lagged weekly count of chickenpox cases.")
print("Each snapshot shows the chickenpox cases (per node) in the upcoming week t+1 as target y.")

The temporal dataset has 517 snapshots.
Each snapshot has 4 features per node. Which shows the lagged weekly count of chickenpox cases.
Each snapshot shows the chickenpox cases (per node) in the upcoming week t+1 as target y.


In [51]:
snapshot

Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])

In [49]:
print(f"The normal dataset has {data.num_temp_features} temporal features per node. Which shows the weekly count of chickenpox cases for the whole time period.")
print(f"Depending on the tasks, the target in the normal dataset can be defined freely.")

The normal dataset has 522 temporal features per node. Which shows the weekly count of chickenpox cases for the whole time period.
Depending on the tasks, the target in the normal dataset can be defined freely.


In other words: The temporal dataset gives us 517 different Graphs which account for the Time Series so that each Graph is similiar to a Time Series Forecasting Matrix where the features are the previous time steps and the taget the upcoming weeks. Since the lag is 4 the first 3 rows cannot be used. But we dont know why there are 5 rows  missing (522 -> 517)

# Task 3: Define a model using pytorch-geometric-temporal

In [52]:
from torch_geometric_temporal import temporal_signal_split

## i. Split the dataset into train and test.

In [ ]:
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)


## ii. Train your model on the training dataset


In [58]:
import torch
torch.manual_seed(42)
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [59]:
model = RecurrentGCN(node_features = 4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in range(200):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat.squeeze() - snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()


## iii. Test your model on the test dataset and evaluate it for MSE.

In [61]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat.squeeze() - snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 0.7201


Since our target here is a continous variable and we are doing a Regression (Forecasting the Chickenpox counts based on previous counts), we use MSE (or RMSE), which sums up the squared differnece between the truth and the predicted value. Since it squares the errors, big differneces are stongly penalized but it is also sensitive to outliers. 

Beacuse we are doing a regression task its better to use MSE or MAE. If we would do a classification task, metrics such as f1-score or accuracy or AUROC could be used.

# Task 4: Set up a hyperparameter optimization pipeline with nested 5-fold cross-validation.

# Task 5: Test implementations of 3 different models (such as GConvLSTM, GConvGRU, STGCN, DCRNN) and find out the best performing model


# Task 6: Compare your results with the following study: https://arxiv.org/abs/2102.08100